In [0]:
# Class IT-533 Week #3  - Wrangling Data: Preprocessing and Visualizing
# Summer 2020
# Lab #2 - UCI_Credit_Card.csv
# By Wei Geng(David) and Anusha Bale
# Created on: 05/31/2020
# Honor Code: “I have neither given or received, nor have I tolerated others' use of unauthorized aid.”

In [2]:
# Convert to pandas DataFrame
pip install pandas
import io
import pandas as pd
credit_card_df = pd.read_csv('https://raw.githubusercontent.com/intelspeedstep/AMOD533/master/UCI_Credit_Card.csv')
credit_card_df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,3102.0,689.0,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,1725.0,2682.0,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,14027.0,13559.0,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,0,0,46990.0,48233.0,49291.0,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,0,0,8617.0,5670.0,35835.0,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [3]:
# output the data attributes and its datatypes, plus the shape of the dataset
print(credit_card_df.info(verbose=True))
print(credit_card_df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          30000 non-null  int64  
 1   LIMIT_BAL                   30000 non-null  float64
 2   SEX                         30000 non-null  int64  
 3   EDUCATION                   30000 non-null  int64  
 4   MARRIAGE                    30000 non-null  int64  
 5   AGE                         30000 non-null  int64  
 6   PAY_0                       30000 non-null  int64  
 7   PAY_2                       30000 non-null  int64  
 8   PAY_3                       30000 non-null  int64  
 9   PAY_4                       30000 non-null  int64  
 10  PAY_5                       30000 non-null  int64  
 11  PAY_6                       30000 non-null  int64  
 12  BILL_AMT1                   30000 non-null  float64
 13  BILL_AMT2                   300

### <font color = 'orange'>Question #1: Data Transformation: Pick three variables whose data types may be more useful as another data type and use R or Python to transform them to numeric or character/ categorical format.  Then, explain in no less than 3 sentences what this transformation will allow you to accomplish analytically later on.</font>
<br> <font color = 'green'>Answer:1) Transforming "sex" from numeric to categorical would allow me to slice and dice the data by comparing "sex" to see if there is disparities between male and female. 2) Transforming "education" from numeric to categorical would allow me to slice and dice the data by comparing education to see if there is disparities between people with different education level. 3) Transforming "marriage" from numeric to categorical would allow me to slice and dice the data by comparing marital status to see if there is disparities between people with different married and single.
</font></br>

In [4]:
# Exam the levels of each variable which I picked --  sex, education and marriage
print(credit_card_df['SEX'].value_counts(normalize=True,dropna=False))
print(credit_card_df['EDUCATION'].value_counts(normalize=True,dropna=False))
print(credit_card_df['MARRIAGE'].value_counts(normalize=True,dropna=False))

2    0.603733
1    0.396267
Name: SEX, dtype: float64
2    0.467667
1    0.352833
3    0.163900
5    0.009333
4    0.004100
6    0.001700
0    0.000467
Name: EDUCATION, dtype: float64
2    0.532133
1    0.455300
3    0.010767
0    0.001800
Name: MARRIAGE, dtype: float64


In [5]:
# I would pick sex, education and marriage to transform them from numeric to categorical(object) using map function
credit_card_df['SEX']=credit_card_df['SEX'].map({1:'male',2:'female'})
credit_card_df['EDUCATION']=credit_card_df['EDUCATION'].map({1:'graduate school',2:'university',3:'high school',4:'others',5:'others',6:'others',0:'others'})
credit_card_df['MARRIAGE']=credit_card_df['MARRIAGE'].map({1:'married',2:'single',3:'others',0:'others'})
credit_card_df[['SEX','EDUCATION','MARRIAGE']].info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   SEX        30000 non-null  object
 1   EDUCATION  30000 non-null  object
 2   MARRIAGE   30000 non-null  object
dtypes: object(3)
memory usage: 703.2+ KB


### <font color = 'orange'>Question #2: Data Substitution: Use R or Python to substitute values in at least two variables. Explain in no less than 3 sentences how you decided what values to use for substitution and, in addition to your explanation, show your code.</font>
<br> <font color = 'green'>Answer: There are two variables I used substitute method to replace values. 1) In the code book, there is no definition or explaination of what 'others' means in the Education attributes. It is better to replace 'others' with 'unknown' because it is very likily that this information was not collected at the data collection level. 2) The same issue we have for the marriage attributes. we dont have clear definition of what 'other' really mean. so, we will replace it with 'unknown'.  </font>


In [6]:
credit_card_df['EDUCATION']=credit_card_df['EDUCATION'].str.replace('others','unknown')
credit_card_df['MARRIAGE']=credit_card_df['MARRIAGE'].str.replace('others','unknown')
print('Values in Education Variable are: ',credit_card_df['EDUCATION'].unique())
print('Values in Marriage Variable are: ',credit_card_df['MARRIAGE'].unique())

Values in Education Variable are:  ['university' 'graduate school' 'high school' 'unknown']
Values in Marriage Variable are:  ['married' 'single' 'unknown']


### <font color = 'orange'>Question #3: Data Reduction: Pick two variables that could be simplified, either by combining two or more attributes (=columns) into one, or by binning or bucketing, then use R or Python to accomplish this.  Explain in no less than 3 sentences what benefit this may provide for any later analysis. </font>
<br> <font color = 'green'>Answer: I pick two variables, BILL_AMT and PAY_AMT to simplify. First, I would average all 6 amount columns, then use the qcut() method to binning the avarege amount at customer level. Last, I output the binning values for BILL_AMT and PAY_AMT. You can see the results in code below.

In [7]:
# Simplify BILL_AMT variables by binning the average of all bill amounts into 10 category values
credit_card_df['BILL_AMT_AVG']=credit_card_df[['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']].mean(axis=1)
credit_card_df['BILL_AMT_AVG_CAT']=pd.qcut(credit_card_df['BILL_AMT_AVG'],10,precision=2)
credit_card_df['BILL_AMT_AVG_CAT'].drop_duplicates().sort_values().reset_index(drop=True)

0       (-56043.18, 697.28]
1         (697.28, 2861.17]
2        (2861.17, 7373.52]
3        (7373.52, 13764.1]
4       (13764.1, 21051.83]
5      (21051.83, 31478.83]
6      (31478.83, 46277.53]
7      (46277.53, 72535.13]
8     (72535.13, 123933.23]
9    (123933.23, 877313.83]
Name: BILL_AMT_AVG_CAT, dtype: category
Categories (10, interval[float64]): [(-56043.18, 697.28] < (697.28, 2861.17] < (2861.17, 7373.52] <
                                     (7373.52, 13764.1] ... (31478.83, 46277.53] <
                                     (46277.53, 72535.13] < (72535.13, 123933.23] <
                                     (123933.23, 877313.83]]

In [8]:
# Simplify PAY_AMT variables by binning the average of all payment amounts into 10 category values
credit_card_df['PAY_AMT_AVG']=credit_card_df[['PAY_AMT1','PAY_AMT2','PAY_AMT3','PAY_AMT4','PAY_AMT5','PAY_AMT6']].mean(axis=1)
credit_card_df['PAY_AMT_AVG_CAT']=pd.qcut(credit_card_df['PAY_AMT_AVG'],10,precision=2)
credit_card_df['PAY_AMT_AVG_CAT'].drop_duplicates().sort_values().reset_index(drop=True)

0          (-0.01, 404.07]
1         (404.07, 913.47]
2        (913.47, 1306.67]
3       (1306.67, 1755.03]
4       (1755.03, 2397.17]
5       (2397.17, 3341.13]
6       (3341.13, 4697.77]
7       (4697.77, 6810.73]
8      (6810.73, 11958.27]
9    (11958.27, 627344.33]
Name: PAY_AMT_AVG_CAT, dtype: category
Categories (10, interval[float64]): [(-0.01, 404.07] < (404.07, 913.47] < (913.47, 1306.67] <
                                     (1306.67, 1755.03] ... (3341.13, 4697.77] < (4697.77, 6810.73] <
                                     (6810.73, 11958.27] < (11958.27, 627344.33]]

### <font color = 'orange'>Question #4: Now that you have a better dataset, list three (3) important organizational analytical questions that management could ask about the dataset in order to help them make business or organizational decisions.  </font>
<br> <font color = 'green'>Answer: Below are the question I think the management would ask: 1) Management team need to better understand our customers, so could you breakdown the customers by demographics such as age, gender, and education? 2) Which is the group that get most average bill amount, and what are their attributes? 3) Which is the group that pay least average pay amount, and is the group who are senior citizens?   </font>